In [1]:
%%html
<style>
 .widget-textarea textarea {
    font-family: Menlo, Monaco, "Courier New", monospace;
    font-size: 14px;
}
</style>

In [2]:
from ipywidgets import Button, Textarea, Layout
from db import db

def find_not_viewed() -> dict:
    return db.repka_logs.find_one({ "is_viewed": { "$ne": True } })

def mark_as_viewed(one):
    db.repka_logs.update_many({"session_id": one['session_id']}, {"$set": { 'is_viewed': True }})


textarea = Textarea(
    layout=Layout(width='800px', height='500px')
)

textarea.style.font_size = '100px'

def print_session(one, textarea):
    text = []

    for one in db.repka_logs.find({ "session_id": one['session_id'] }).sort([("message_id", 1)]):
        text.append(f"- {one['request']}\n- {one['response']}\n\n")

    textarea.value = "".join(text)


submit = Button(description="Посмотрел", button_style='success')
one = None

def on_button_clicked(b):
    global one

    if one: 
        mark_as_viewed(one)

    one = find_not_viewed()
    print_session(one, textarea)
    b.description = f"X {one['session_id']}"

submit.on_click(on_button_clicked)

display(submit)
display(textarea)

Button(button_style='success', description='Посмотрел', style=ButtonStyle())

Textarea(value='', layout=Layout(height='500px', width='800px'))

In [3]:
db.command({ 'collStats' : "repka_logs", 'scale': 1024*1024 })

{'ns': 'skills.repka_logs',
 'size': 22,
 'count': 90386,
 'avgObjSize': 263,
 'storageSize': 8,
 'freeStorageSize': 0,
 'capped': False,
 'wiredTiger': {'metadata': {'formatVersion': 1},
  'creationString': 'access_pattern_hint=none,allocation_size=4KB,app_metadata=(formatVersion=1),assert=(commit_timestamp=none,durable_timestamp=none,read_timestamp=none,write_timestamp=off),block_allocation=best,block_compressor=snappy,cache_resident=false,checksum=on,colgroups=,collator=,columns=,dictionary=0,encryption=(keyid=,name=),exclusive=false,extractor=,format=btree,huffman_key=,huffman_value=,ignore_in_memory_cache_size=false,immutable=false,import=(enabled=false,file_metadata=,repair=false),internal_item_max=0,internal_key_max=0,internal_key_truncate=true,internal_page_max=4KB,key_format=q,key_gap=10,leaf_item_max=0,leaf_key_max=0,leaf_page_max=32KB,leaf_value_max=64MB,log=(enabled=false),lsm=(auto_throttle=true,bloom=true,bloom_bit_count=16,bloom_config=,bloom_hash_count=8,bloom_oldest=fa

In [4]:
from IPython.display import display
